# Music generation with LSTM Network: generation

In this part, we are trying to generate new music tracks from the model trained. The generation process includes some steps already explained in the training part. These steps are the mostly related to the data preparation.

## I- Import the necessary libraries:

In [0]:
""" This module generates notes for a midi file using the
    trained neural network """
import pickle
import numpy
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Activation
from keras.models import load_model
import keras

## II- Data preparation:

The process of preparing sequences is the same as in the training part. We introduce the notes, which can be the ones on which we trained the model if we want output with the same notes' dictionary. However, if we want to generate music from a specific set of notes in some tracks, we can introduce some new midi files and prepare a new dictionary. The sequence length used in here is 100.



In [0]:
def prepare_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

## III- Newtwork preparation:

Training LSTM deep learning models is computationally intensive. For instance, Training the model we use for prediction on the kaggle platform with a 16 Go GPU took 9 hours to train a model on 62 epochs. Therefore, we prepare a model with the same structure (same layers and number of neurons), and upload the weigths which we saved in the training part.

In [0]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights('../input/weights/weights-improvement-62-1.0400-bigger.hdf5')

    return model

## IV- Notes generation:

The process of notes generation is based on predicting the most probable next note from the dictionary. 

In fact, the generation function predicts a 500 sequence of notes. The first step is to take a random starting point from the notes' dictionary introduced. After that, we predict one note at a time.

The prediction takes the argmax of probabilities between the possible outputs, which means the class (ie the note) with the highest probability. 

Each time, the note is added to the ones we already predicted and the sequence we introduce for the next prediction is updated to introduce the latest note we got. At the end, we have a list with 500 notes generated.

In [0]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output


## V- Audio file creation:

We get the notes predicted from the function generate_notes, the output is a list containing chords and notes in the form of a string. So, in order to transfrorm our prediction output to an audio file, we use the functions that are already built in the music21 library.

The issue in this part that is encountered is to generate notes with instruments other than the Piano. According to the documentation of the library, the use of the [instrument."Name of the instrument"] function determines on which instrument the notes are generated. However, when we tried to generate notes with instruments other than piano, for instance AcousticGuitar, Clarinet, Drums..., the output is always piano notes.

After generating the notes one by one, and seperating them with an offset of 0.5, we use the function .Stream to put all the output in an audio file that we save in a midi file.

In [0]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
                new_chord = chord.Chord(notes)
                new_chord.offset = offset
                output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
           # new_note1 = note.Note(pattern)
           # new_note1.offset = offset
            #new_note1.storedInstrument = instrument.Clarinet
            output_notes.append(new_note)
            #output_notes.append(new_note1)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output_dru.mid')


This function is used to group all the functions presented above.

In [0]:

def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('../input/notesmusic/fnotes_instruments', 'rb') as filepath:
        notes = pickle.load(filepath)
        notes = notes[5][1:]   #guitar notes
        notes.append('0.1.1')
    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    #model = load_model('weights-02-4.7661-dataall.hdf5')
   # model = load_model('weights-02-4.7661-dataall.hdf5', custom_objects={
    #'Adam': lambda **kwargs: hvd.DistributedOptimizer(keras.optimizers.Adam(**kwargs))})
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)



                
        
        
        
        